# 🔧 GSE182159 배치 파일 통합 (메모리 효율 버전)

**문제**: 전체 배치를 한 번에 로드하면 RAM 부족

**해결**: 2개씩 단계별 통합

| 단계 | 작업 |
|------|------|
| 1 | batch_0 + batch_1 → merged_01 |
| 2 | batch_2 + batch_3 → merged_23 |
| 3 | merged_01 + merged_23 + batch_4 → final |

In [ ]:
# @title STEP 1: 환경 설정
from google.colab import drive
drive.mount('/content/drive')

!pip install anndata pandas scipy -q

import anndata as ad
import pandas as pd
import numpy as np
import scipy.sparse as sp
import os
import gc

RAW_PATH = '/content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data'

# 배치 파일 확인
batch_files = sorted([f for f in os.listdir(RAW_PATH) if f.startswith('GSE182159_batch_')])
print(f'📦 배치 파일: {len(batch_files)}개')
for bf in batch_files:
    size_gb = os.path.getsize(f'{RAW_PATH}/{bf}') / 1e9
    print(f'   {bf}: {size_gb:.2f} GB')

print('\n✅ 환경 설정 완료!')

In [ ]:
# @title STEP 2a: batch_0 + batch_1 통합
print('📦 batch_0 + batch_1 통합 중...')

adata_0 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_batch_0.h5ad')
print(f'   batch_0: {adata_0.n_obs:,} cells')

adata_1 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_batch_1.h5ad')
print(f'   batch_1: {adata_1.n_obs:,} cells')

# 통합
merged_01 = ad.concat([adata_0, adata_1], join='outer')
merged_01.var_names_make_unique()

# sparse matrix로 변환 (메모리 절약)
if not sp.issparse(merged_01.X):
    merged_01.X = sp.csr_matrix(merged_01.X)

# 저장
merged_01.write_h5ad(f'{RAW_PATH}/GSE182159_merged_01.h5ad')
print(f'\n✅ merged_01 저장: {merged_01.n_obs:,} cells')

# 메모리 정리
del adata_0, adata_1, merged_01
gc.collect()
print('🧹 메모리 정리 완료')

In [ ]:
# @title STEP 2b: batch_2 + batch_3 통합
print('📦 batch_2 + batch_3 통합 중...')

adata_2 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_batch_2.h5ad')
print(f'   batch_2: {adata_2.n_obs:,} cells')

adata_3 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_batch_3.h5ad')
print(f'   batch_3: {adata_3.n_obs:,} cells')

# 통합
merged_23 = ad.concat([adata_2, adata_3], join='outer')
merged_23.var_names_make_unique()

# sparse matrix로 변환
if not sp.issparse(merged_23.X):
    merged_23.X = sp.csr_matrix(merged_23.X)

# 저장
merged_23.write_h5ad(f'{RAW_PATH}/GSE182159_merged_23.h5ad')
print(f'\n✅ merged_23 저장: {merged_23.n_obs:,} cells')

# 메모리 정리
del adata_2, adata_3, merged_23
gc.collect()
print('🧹 메모리 정리 완료')

In [ ]:
# @title STEP 3: 최종 통합 (merged_01 + merged_23 + batch_4)
print('📦 최종 통합 중...')
print('   (런타임 재시작 후 이 셀부터 실행해도 됩니다)\n')

# 필요시 다시 import
import anndata as ad
import pandas as pd
import scipy.sparse as sp
import os
import gc

RAW_PATH = '/content/drive/MyDrive/tahoe-x1/viral_hepatitis/raw_data'

# merged 파일들 로드
print('   Loading merged_01...')
merged_01 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_merged_01.h5ad')
print(f'   ✅ merged_01: {merged_01.n_obs:,} cells')

print('   Loading merged_23...')
merged_23 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_merged_23.h5ad')
print(f'   ✅ merged_23: {merged_23.n_obs:,} cells')

print('   Loading batch_4...')
batch_4 = ad.read_h5ad(f'{RAW_PATH}/GSE182159_batch_4.h5ad')
print(f'   ✅ batch_4: {batch_4.n_obs:,} cells')

# 최종 통합
print('\n🔗 최종 통합 중...')
adata = ad.concat([merged_01, merged_23, batch_4], join='outer')
adata.var_names_make_unique()

# sparse matrix 확인
if not sp.issparse(adata.X):
    adata.X = sp.csr_matrix(adata.X)

print(f'\n✅ 최종 통합 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')

# 메모리 정리
del merged_01, merged_23, batch_4
gc.collect()

In [ ]:
# @title STEP 4: 메타데이터 매핑
metadata_file = f'{RAW_PATH}/sample_metadata.csv'

if os.path.exists(metadata_file):
    print('📋 메타데이터 로드 중...')
    metadata_df = pd.read_csv(metadata_file)
    print(f'   샘플 수: {len(metadata_df)}')
    
    # GSM ID 추출
    adata.obs['gsm_id'] = adata.obs['sample'].str.extract(r'(GSM\d+)')[0]
    
    # 매핑
    meta_indexed = metadata_df.set_index('sample_id')
    for col in metadata_df.columns:
        if col != 'sample_id':
            adata.obs[col] = adata.obs['gsm_id'].map(meta_indexed[col].to_dict())
    
    print(f'\n📊 HBV Stage 분포:')
    if 'Stage' in adata.obs.columns:
        print(adata.obs['Stage'].value_counts())
    
    print('\n✅ 메타데이터 매핑 완료!')
else:
    print('⚠️ 메타데이터 파일이 없습니다.')

In [ ]:
# @title STEP 5: 최종 파일 저장
output_file = f'{RAW_PATH}/GSE182159_processed.h5ad'

print(f'💾 저장 중: {output_file}')
print('   (이 작업은 몇 분이 소요될 수 있습니다)\n')

adata.write_h5ad(output_file)

print('=' * 60)
print('🎉 GSE182159 데이터 통합 완료!')
print('=' * 60)
print(f'\n📁 저장된 파일: {output_file}')
print(f'\n📊 데이터 요약:')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   유전자 수: {adata.n_vars:,}')
print(f'   샘플 수: {adata.obs["sample"].nunique()}')
print(f'\n📋 obs 컬럼: {list(adata.obs.columns)}')
print('\n✅ 다음 단계: Step1_HBV_real.ipynb 실행')

In [ ]:
# @title (선택) 중간 파일 정리
# 최종 파일 저장 확인 후 실행

cleanup = False  # @param {type:"boolean"}

if cleanup:
    import os
    merged_files = ['GSE182159_merged_01.h5ad', 'GSE182159_merged_23.h5ad']
    for mf in merged_files:
        path = f'{RAW_PATH}/{mf}'
        if os.path.exists(path):
            os.remove(path)
            print(f'🗑️ 삭제: {mf}')
    print('\n✅ 중간 파일 정리 완료')
else:
    print('⏭️ 정리 건너뜀 (cleanup=True로 변경하여 실행)')